In [151]:
import astropy.units as u
# Function to create image array stacks from given timestamps and locations
def makeStacks(path,timeStamps,channels=['94','131','171','193','211','304','335'],initCoords=[0*u.arcsec,0*u.arcsec],ROIsize=300*u.arcsec,rotationCorr=1,threads=1,clusterScale=16,addNoise=0,level1Noise=0):
    ### Create the cluster
    import dask
    import distributed
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster(threads_per_worker=threads)
    cluster.scale(clusterScale)
    client = Client(cluster)
    
    import astropy.units as u
    import sunpy.map
    import time
    import numpy as np
    import drms
    from astropy.coordinates import SkyCoord
    from sunpy.util.metadata import MetaDict
    import sunpy.io.fits
    from sunpy.map import Map
    import os 
    from sunpy.physics import solar_rotation
    import astropy.time
    from astropy.visualization import ImageNormalize, SqrtStretch, time_support
    from aiapy.calibrate import correct_degradation
    from aiapy.calibrate.util import get_correction_table
    from aiapy.calibrate import register, update_pointing
    
    ### Get correction table
    import warnings
    warnings.filterwarnings('ignore')
    correction_table = get_correction_table(correction_table='/ssw/sdo/aia/response/aia_V8_20171210_050627_response_table.txt')
    import warnings
    warnings.filterwarnings('default')
    
    ### Get images
    client_drms = drms.Client()
    keys_aia_tot = []
    paths_aia_tot = []
    for j in range(len(timeStamps[:])):
        keys_aia = []
        paths_aia = []
        for i in range(len(channels)):
          keys,paths = client_drms.query(
              'aia.lev1_euv_12s['+timeStamps[j]+']['+channels[i]+']',
              key=drms.const.all,
              seg='image',
          )
          keys_aia.append(keys)
          paths_aia.append(paths)
        keys_aia_tot.append(keys_aia)
        paths_aia_tot.append(paths_aia)
    
    ### Define the Dask function for distributed computing
    def getFrame(args):

      i = args[0]
      timeStamp = args[1]
      x0 = args[2][0]
      y0 = args[2][1]
      ROIsize = args[3]
      correction_table = args[4]
      path = args[5]
      keys = args[6]
      paths = args[7]
      m_aia_init = args[8]
      rotationCorr = args[9]
      addNoise = args[10]
      level1Noise = args[11]
        
      stack= []
      noisey_stack = []
      level_1_noise = []
      for j in range(len(channels)):
          m_aia = sunpy.map.Map(paths[i][j]['image'][0])
        
          m_normalized_aia = sunpy.map.Map(
            m_aia.data/m_aia.exposure_time.to(u.s).value,
            m_aia.meta
          )
            
          # Create noisey map
          if addNoise == 1:
              noise_data = m_aia.data
              noise_meta = m_aia.meta
              # Add poisson noise for positves and negatives
              np.random.seed(0)
              noise_data = noise_data+(np.abs(noise_data)-np.random.poisson(np.abs(noise_data)))
              m_noise_aia = sunpy.map.Map(noise_data,noise_meta)
              m_normalized_noise_aia = sunpy.map.Map(
                m_noise_aia.data/m_noise_aia.exposure_time.to(u.s).value,
                m_noise_aia.meta
              )

              # Isolate level 1 noise for analysis
              if level1Noise == 1:
                  level_1_placeholder = m_normalized_aia
                  level_1_noise_placeholder = m_normalized_noise_aia    
            
          # Back to original map
          m_updated_pointing_aia = update_pointing(m_normalized_aia)
          m_registered_aia = register(m_updated_pointing_aia)
          m_corrected_aia = correct_degradation(m_registered_aia,correction_table=correction_table)
          maps_frame_aia = m_corrected_aia
          #maps_frame_aia = m_registered_aia
          ###
          # Adjust for shift in images and apply it to x0 and y0
          ###
          if rotationCorr == 1:
              adjustMaps = sunpy.map.Map(m_aia_init,maps_frame_aia,sequence=True)
              adjust = sunpy.physics.solar_rotation.calculate_solar_rotate_shift(adjustMaps)
              x0 = x0 - adjust['x'][1]
              y0 = y0 - adjust['y'][1]
          ###
          bottom_left_aia = SkyCoord(x0 - ROIsize, y0 - ROIsize,
                               frame=maps_frame_aia.coordinate_frame)
          top_right_aia = SkyCoord(x0 + ROIsize, y0 + ROIsize,
                             frame=maps_frame_aia.coordinate_frame)
          submap_frame_aia = maps_frame_aia.submap(bottom_left_aia,top_right_aia)
                                                                                    
          # Back to noisey map    
          if addNoise == 1:
              m_updated_pointing_noise_aia = update_pointing(m_normalized_noise_aia)
              m_registered_noise_aia = register(m_updated_pointing_noise_aia)
              m_corrected_noise_aia = correct_degradation(m_registered_noise_aia,correction_table=correction_table)
              maps_frame_noise_aia = m_corrected_noise_aia

              bottom_left_noise_aia = SkyCoord(x0 - ROIsize, y0 - ROIsize,
                                   frame=maps_frame_noise_aia.coordinate_frame)
              top_right_noise_aia = SkyCoord(x0 + ROIsize, y0 + ROIsize,
                                 frame=maps_frame_noise_aia.coordinate_frame)
              submap_frame_noise_aia = maps_frame_noise_aia.submap(bottom_left_noise_aia,top_right_noise_aia)

              # Create submaps for level 1 noise, we are doing this down here so x0 and y0 can be updated if rotation is turned on
              if level1Noise == 1:
                  xScale = level_1_placeholder.scale[0].value
                  yScale = level_1_placeholder.scale[1].value
                  xCenterPixel = np.floor(x0.value/xScale)+(level_1_placeholder.data.data.shape[0]/2)
                  yCenterPixel = np.floor(y0.value/yScale)+(level_1_placeholder.data.data.shape[0]/2)
                  # Cut to same size across channels so they can be stacked, the actual arcsec FOV doesn't matter since this will be used for a histogram  
                  cutSizeX = submap_frame_aia.data.shape[0]
                  cutSizeY = submap_frame_aia.data.shape[1]
                  if cutSizeX%2 == 1:
                    leftX = xCenterPixel-(cutSizeX-1)/2
                    rightX = xCenterPixel+(cutSizeX-1)/2+1
                  else:
                    leftX = xCenterPixel-(cutSizeX)/2
                    rightX = xCenterPixel+(cutSizeX)/2
                  if cutSizeY%2 == 1:
                    bottomY = yCenterPixel-(cutSizeY-1)/2
                    topY = yCenterPixel+(cutSizeY-1)/2+1
                  else:
                    bottomY = yCenterPixel-(cutSizeY)/2
                    topY = yCenterPixel+(cutSizeY)/2
                  lvl1_submap = level_1_placeholder.data[int(leftX):int(rightX),int(bottomY):int(topY)]
                  lvl1_noise_submap = level_1_noise_placeholder.data[int(leftX):int(rightX),int(bottomY):int(topY)]

          # Compile stacks
          if addNoise == 1:
              noisey_stack.append(submap_frame_noise_aia.data)
              if level1Noise == 1:
                  level_1_noise.append(lvl1_noise_submap-lvl1_submap)  
          stack.append(submap_frame_aia.data)
      if addNoise == 1 and level1Noise == 1:
          return np.stack(stack),np.stack(noisey_stack),np.stack(level_1_noise) 
      elif addNoise == 1:
          return np.stack(stack),np.stack(noisey_stack);
      else:
          return np.stack(stack)
      
    ### Create loop for Dask function
    m_aia_init0 = sunpy.map.Map(paths_aia_tot[0][0]['image'][0])
    m_updated_pointing_aia_init = update_pointing(m_aia_init0)
    m_aia_init  = register(m_updated_pointing_aia_init)
    lazy_results = []
    for i in range(len(timeStamps)):
      args = [i,timeStamps[i],initCoords,ROIsize,correction_table,path,keys_aia_tot,paths_aia_tot,m_aia_init,rotationCorr,addNoise,level1Noise]
      lazy = dask.delayed(getFrame)(args)
      lazy_results.append(lazy)
    ### Execute function
    stacks = dask.compute(lazy_results)
    client.close()
    cluster.close()
    if addNoise == 1 and level1Noise == 1:
        return np.squeeze(np.stack(stacks[0],axis=1))[0],np.squeeze(np.stack(stacks[0],axis=1))[1],np.squeeze(np.stack(stacks[0],axis=1))[2]
    elif addNoise == 1:
        return np.squeeze(np.stack(stacks[0],axis=1))[0],np.squeeze(np.stack(stacks[0],axis=1))[1];
    else:
        return np.stack(stacks[0],axis=0);

In [152]:
###
# You need aiapy, sunpy, graphviz instaled
###
import astropy.units as u
import os

import warnings
warnings.filterwarnings('ignore')

path = '/Users/penwarden/Desktop/'

### Get training data
addNoise = 1
tStart = ''
tLength = '1h'
tCadence = '1440s'
trainStamps = ['2010-07-15T00:00:00','2010-07-16T00:00:00','2010-06-14T00:00:00','2010-06-15T00:00:00']
rotationCorr = 0
ROIsize=500*u.arcsec
initCoords = [51*u.arcsec,248*u.arcsec]
#threads = 1
#clusterScale = 16
import warnings
warnings.filterwarnings('ignore')
trainStack, trainNoiseStack = makeStacks(path,trainStamps,initCoords=initCoords,rotationCorr=rotationCorr,ROIsize=ROIsize,addNoise=addNoise)
import warnings
warnings.filterwarnings('default')

### Get recent testing data without noise
initCoords = [-44*u.arcsec,-49*u.arcsec]
initCoords = [-400*u.arcsec,-400*u.arcsec]
testStamps = ['2017-08-26T02:52:58','2018-08-26T00:00:00'] # AR & QS
import warnings
warnings.filterwarnings('ignore')
testStack2017 = makeStacks(path,testStamps,initCoords=initCoords,rotationCorr=rotationCorr,ROIsize=ROIsize)
import warnings
warnings.filterwarnings('default')

### Get old testing data with noise
initCoords = [83*u.arcsec,126*u.arcsec]
testStamps = ['2010-08-15T00:00:00','2011-01-28T00:00:00'] # AR & QS
addNoise = 1
level1Noise = 1
import warnings
warnings.filterwarnings('ignore')
testStack, testNoiseStack, level_1_testNoise = makeStacks(path,testStamps,initCoords=initCoords,rotationCorr=rotationCorr,ROIsize=ROIsize,addNoise=addNoise,level1Noise = level1Noise)
import warnings
warnings.filterwarnings('default')

In [160]:
# Plotting function (2 images with 3 sets of channels each)
def stackPlotter(names,stacks,file_name,histName,saveFigure,bins,pathDir,testType = 1,channels=['$_{94}$','$_{131}$','$_{171}$','$_{193}$','$_{211}$','$_{304}$','$_{335}$'],cmaps=['sdoaia94','sdoaia131','sdoaia171','sdoaia193','sdoaia211','sdoaia304','sdoaia335'],lossWeights = [1,1,1,1,1,1,1]):
    import matplotlib.pyplot as plt
    font = {'fontname':'Helvetica'}
    cmapNoise = ['seismic','seismic','seismic','seismic','seismic','seismic','seismic']
    chanNum = len(channels)
    plt.figure(figsize=(5*len(channels),6*(len(names)+2)));
    histColor = ['blue','orange','green']
    annotate = np.empty((2,chanNum,3),dtype='U100')
    for i in range(len(names)):
        for j in range(len(channels)): 
            histCount = int(np.floor(i/3)) # index for which 3 rows go to which histogram
            
            mean = np.mean(stacks[i][j])
            std = np.std(stacks[i][j])
            median = np.median(stacks[i][j])
            centerThree = (histCount)*3+1 # center index of groups of three
            listMean = [np.mean(stacks[centerThree-1][j]),np.mean(stacks[centerThree][j]),np.mean(stacks[centerThree+1][j])]
            listSTD = [np.std(stacks[centerThree-1][j]),np.std(stacks[centerThree][j]),np.std(stacks[centerThree+1][j])]
            xmax = np.max(np.concatenate([stacks[centerThree-1][j].flatten(),stacks[centerThree][j].flatten(),stacks[centerThree+1][j].flatten()]))
            xmin = np.min(np.concatenate([stacks[centerThree-1][j].flatten(),stacks[centerThree][j].flatten(),stacks[centerThree+1][j].flatten()]))
            
            plt.subplot(len(names)+2,chanNum,i*chanNum+histCount*chanNum+j+1)
            if testType == 0 and i == centerThree+1:
                plt.imshow(stacks[i][j],cmap=cmapNoise[j],vmin = -5, vmax = 5,origin = 'lower') # Set abs(vmin) = vmax to center cmap at 0
            else:
                plt.imshow(stacks[i][j],cmap=cmaps[j],vmin = xmin,vmax = xmax,origin = 'lower')
            plt.title(names[i]+' '+channels[j],fontsize=25, **font)
            plt.xlabel('Pixel',fontsize=10, **font)
            plt.ylabel('Pixel',fontsize=10, **font);

            plt.subplot(len(names)+2,chanNum,(histCount+1)*3*chanNum+j+1+chanNum*histCount)
            plt.hist(stacks[i][j].flatten(),bins=bins,alpha=0.5,density=True,range=[xmin,xmax])
            plt.title('Histograms',fontsize=25, **font)
            plt.xlabel('Pixel Value',fontsize=10),plt.ylabel('Count',fontsize=10, **font);
            plt.gca().set_yscale("log");
            annotate[histCount][j][int(i%3)] = histName[int(i%3)]+'\nmean: '+str(round(mean,2))+'\nstd: '+str(round(std,2))+'\nmedian: '+str(round(median,2));
            plt.xlim([np.max(listMean)-7*np.max(listSTD),np.max(listMean)+21*np.max(listSTD)]);
    # Add legends
    for i in range(0,2):
        for j in range(len(channels)):
            plt.subplot(len(names)+2,chanNum,(i+1)*3*chanNum+j+1+chanNum*i)
            plt.legend([annotate[i][j][0],annotate[i][j][1],annotate[i][j][2]])
    if saveFigure == 1:
        plt.savefig(pathDir+'/'+file_name+'.png');
    plt.close()
    #plt.show()

In [161]:
import numpy as np
def shapeStack (stack,scaleStack):
    shapeInput = [len(stack),stack[0].shape[0]*stack[0].shape[1]]
    scaled = []
    for i in range(len(stack)):
        scaled.append(stack[i]/np.median(scaleStack[i]))
    scaledData = np.transpose(np.reshape(np.stack(scaled),shapeInput))
    
    return scaled, scaledData

In [162]:
# Set the data used for scaling
scaleStack = np.mean(trainStack,0)
#np.save('scaleStack',scaleStack)

# Get train data in correct NN format
cleanTrain = [];cleanTrainData = [];noiseTrain = [];noiseTrainData =[]
for i in range(trainStack.shape[0]):
    clean, cleanData = shapeStack(trainStack[i],scaleStack)
    noise, noiseData = shapeStack(trainNoiseStack[i],scaleStack)
    
    cleanTrain.append(clean),cleanTrainData.append(cleanData),noiseTrain.append(noise),noiseTrainData.append(noiseData) 
    
# Define NN shape
NNshape = [cleanTrainData[0].shape[0]*len(cleanTrainData),cleanTrainData[0].shape[1]]

cleanTrain_NN = np.reshape(np.stack(cleanTrainData),NNshape)
noiseTrain_NN = np.reshape(np.stack(noiseTrainData),NNshape)
cleanTrain_image = np.stack(cleanTrain)
noiseTrain_image = np.stack(noiseTrain)

# Get test data in correct NN format
cleanTest = [];cleanTestData = [];noiseTest = [];noiseTestData =[]
for i in range(testStack.shape[0]):
    clean, cleanData = shapeStack(testStack[i],scaleStack)
    noise, noiseData = shapeStack(testNoiseStack[i],scaleStack)
    
    cleanTest.append(clean),cleanTestData.append(cleanData),noiseTest.append(noise),noiseTestData.append(noiseData) 

cleanTest_NN = np.stack(cleanTestData)
noiseTest_NN = np.stack(noiseTestData)
cleanTest_image = np.stack(cleanTest)
noiseTest_image = np.stack(noiseTest)

# Get recent test data in correct NN format
cleanTestRecent = [];cleanTestRecentData = []
for i in range(testStack.shape[0]):
    clean, cleanData = shapeStack(testStack2017[i],scaleStack)
    cleanTestRecent.append(clean),cleanTestRecentData.append(cleanData)

cleanTestRecent_NN = np.stack(cleanTestRecentData)
cleanTestRecent_image = np.stack(cleanTestRecent)

In [164]:
def createModel(layers,activationFunc,regCoeff):
    # Set seeds for consitant results
    seed_value= 3
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    import random
    random.seed(seed_value)
    import numpy as np
    np.random.seed(seed_value)
    import tensorflow as tf
    tf.random.set_seed(seed_value)

    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

    import keras
    from keras.layers import Input, Dense, Dropout, LeakyReLU, Add
    from keras.models import Model

    ### Create Autoencoder model
    input_pixel = Input(shape=(7,))
    
    if activationFunc == 'LeakyReLU':
        layer = Dense(units=7,activation=LeakyReLU(alpha=0.1),kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(input_pixel)
        for i in range(len(layers)):
            layer = Dense(units=layers[i],activation=LeakyReLU(alpha=0.1),kernel_initializer='glorot_uniform',
                bias_initializer='zeros',activity_regularizer=keras.regularizers.l1(regCoeff))(layer)
        
    elif activationFunc == 'Linear':
        layer = Dense(units=7,kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(input_pixel)
        for i in range(len(layers)):
            layer = Dense(units=layers[i],kernel_initializer='glorot_uniform',
                bias_initializer='zeros',activity_regularizer=keras.regularizers.l1(regCoeff))(layer)
        
    else:
        layer = Dense(units=7,activation='relu',kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(input_pixel)
        for i in range(len(layers)):
            layer = Dense(units=layers[i],activation='relu',kernel_initializer='glorot_uniform',
                bias_initializer='zeros',activity_regularizer=keras.regularizers.l1(regCoeff))(layer)

    output = Dense(units=7,kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(layer)
    
    merge = Add()([output,input_pixel])
    autoencoder = keras.Model(input_pixel,merge)
    autoencoder.summary()
    return autoencoder

In [165]:
def trainModel(autoencoder,trainIn,trainOut,lossWeights,batch_size,epochs,lossFunction,learningRate,validation_split=0.2):
    import keras
    import keras.backend as K  
    # Define custom MSE loss function with weights on channels
    def customLossWrapper(lossWeights,lossFunction):
        def customLoss(yTrue,yPred):
            if lossFunction == 'MSE':
                return K.mean(K.square(yTrue-yPred)*lossWeights,axis=-1) #MSE
            elif lossFunction =='L1':
                return K.mean(K.abs(yTrue-yPred)*lossWeights,axis=-1) #L1
        return customLoss
    
    # Train model
    optimizer = keras.optimizers.Adam(lr = learningRate)
    #optimizer = keras.optimizers.SGD(lr = learningRate, decay=1e-6, momentum=0.9, nesterov=True)
    #autoencoder.compile(optimizer=optimizer,loss=customLossWrapper(lossWeights,lossFunction))
    autoencoder.compile(optimizer=optimizer,loss=keras.losses.MeanAbsoluteError())
    
    history = autoencoder.fit(trainIn,trainOut,epochs=epochs,batch_size=batch_size,validation_split=validation_split,shuffle=True)
    return autoencoder

In [166]:
def testModelArtificial(autoencoder,cleanTest_NN,cleanTest_image,noiseTest_NN,noiseTest_image):
    # Create shape to revert from NN format to image format
    shapeOutput = [len(testStack[0]),testStack[0][0].shape[0],testStack[0][0].shape[1]]

    # Use trained model to denoise test images
    denoiseTest_image = []
    predictedTest = autoencoder.predict(noiseTest_NN[0])
    denoiseTest_image.append(np.reshape(np.transpose(predictedTest),shapeOutput))
    predictedTest = autoencoder.predict(noiseTest_NN[1])
    denoiseTest_image.append(np.reshape(np.transpose(predictedTest),shapeOutput))
    denoiseTest_image = np.stack(denoiseTest_image)

    # Isolate added and removed noise
    denoiseOnly = []
    denoiseOnly.append(np.stack(noiseTest_image)[0]-denoiseTest_image[0])
    denoiseOnly.append(np.stack(noiseTest_image)[1]-denoiseTest_image[1])
    denoiseOnly = np.stack(denoiseOnly)

    noiseOnly = []
    noiseOnly.append(np.stack(noiseTest_image)[0]-np.stack(cleanTest_image)[0])
    noiseOnly.append(np.stack(noiseTest_image)[1]-np.stack(cleanTest_image)[1])
    noiseOnly = np.stack(noiseOnly)
    return denoiseOnly, noiseOnly, denoiseTest_image

In [167]:
def testModelUnmodified(autoencoder,cleanTest_NN,cleanTest_image):
    # Create shape to revert from NN format to image format
    shapeOutput = [len(testStack[0]),testStack[0][0].shape[0],testStack[0][0].shape[1]]

    # Use trained model to denoise test images
    denoiseTest_image = []
    predictedTest = autoencoder.predict(cleanTest_NN[0])
    denoiseTest_image.append(np.reshape(np.transpose(predictedTest),shapeOutput))
    predictedTest = autoencoder.predict(cleanTest_NN[1])
    denoiseTest_image.append(np.reshape(np.transpose(predictedTest),shapeOutput))
    denoiseTest_image = np.stack(denoiseTest_image)

    # Isolate added and removed noise
    denoiseOnly = []
    denoiseOnly.append(np.stack(cleanTest_image)[0]-denoiseTest_image[0])
    denoiseOnly.append(np.stack(cleanTest_image)[1]-denoiseTest_image[1])
    denoiseOnly = np.stack(denoiseOnly)
    
    return denoiseOnly, denoiseTest_image

In [168]:
# Undo the scale applied for the NN
def rescale (stack,scaleStack):
    newStack = []
    for i in range(len(stack)):
        newStack.append(stack[i]*np.median(scaleStack[i]))
    return np.stack(newStack)

In [169]:
def chiSquare (data_1,data_2,labels,title,bins=100):
    # Get hist values for chi square
    hist_1, bin_edges_1 = np.histogram(data_1,bins=bins,density=True,range=[data_1.mean()-3*data_1.std(),data_1.mean()+3*data_1.std()])
    hist_2, bin_edges_1 = np.histogram(data_2,bins=bins,density=True,range=[data_2.mean()-3*data_2.std(),data_2.mean()+3*data_2.std()])
              # Chi square Test
    s, p = scipy.stats.chisquare(hist_1,hist_2)
    plt.legend(labels);
    plt.ylabel('Count')
    plt.xlabel('Pixel Value');
    plt.title(title+': '+'Chi2 = '+str(s)+' & p = '+str(p));
    plt.show()

In [170]:
def getChi2(data_1,data_2):
    import scipy
    #d1 = data_1.flatten()
    #d2 = data_2.flatten()
    min_1 = data_1.mean()-3*data_1.std()
    max_1 = data_1.mean()+3*data_1.std()
    min_2 = data_2.mean()-3*data_2.std()
    max_2 = data_2.mean()+3*data_2.std()
    minimum = np.min([min_1,min_2])
    maximum = np.max([max_1,max_2])
    width = np.abs(minimum-maximum)
    bins = int(np.ceil(width/0.25)) # Accuracy should be no greater then 0.25 DN because of quantization
    hist_1, bin_edges_1 = np.histogram(data_1,bins=bins,density=True,range=[min_1,max_1])
    hist_2, bin_edges_2 = np.histogram(data_2,bins=bins,density=True,range=[min_2,max_2])
    s, p = scipy.stats.chisquare(hist_1,hist_2)
    #chi2 = np.sum(((d1-d2)**2)/d2)
    return s

In [171]:
def colorDataFrameArtificial(x):
    
    c1 = 'background-color: forestgreen'
    c2 = 'background-color: firebrick'
    c = 'color: black'
    cg = 'background-color: lightgrey'
    cb = 'background-color: white'

    # M2_D
    m1_8 = (x.iloc[:,8] <= x.iloc[:,4]) 
    m2_8= (x.iloc[:,8] > x.iloc[:,4])

    # M1_rN
    m1_11 = (x.iloc[:,11] >= -0.05) & (x.iloc[:,11] <= 0.05)
    m2_11 = (x.iloc[:,11] < -0.05) | (x.iloc[:,11] > 0.05)

    # M2_rN
    m1_12 = (x.iloc[:,12] >= x.iloc[:,10]*0.8) & (x.iloc[:,12] <= x.iloc[:,10]*1.2)
    m2_12 = (x.iloc[:,12] < x.iloc[:,10]*0.8) | (x.iloc[:,12] > x.iloc[:,10]*1.2)

    df = pd.DataFrame(cg,index=x.index,columns=x.columns)

    df.iloc[:,8] = np.select([m1_8,m2_8],[c1,c2],default=cb)
    df.iloc[:,11] = np.select([m1_11,m2_11],[c1,c2],default=cb)
    df.iloc[:,12] = np.select([m1_12,m2_12],[c1,c2],default=cb)

    return df

In [172]:
def colorDataFrameUnmodified(x):
    cg = 'background-color: lightgrey'
    df = pd.DataFrame(cg,index=x.index,columns=x.columns)
    return df

In [173]:
def runTraining(lossFunc,activationFunc,lossWeight,hiddenLayers,learningRate,batch_size,epochs,regCoeff,path):
    import pandas as pd
    # Clear Prior Model
    import keras
    keras.backend.clear_session()
    # Create Model
    model = createModel(hiddenLayers,activationFunc,regCoeff)
    # Train Model
    trainedModel = trainModel(model,noiseTrain_NN,cleanTrain_NN,lossWeight,batch_size,epochs,lossFunc,learningRate)
    
    # Save trained model
    trainedModel.save(path+'/trainedModel')
    
    return trainedModel

In [174]:
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)
def runTestingArtificial(trainedModel,lossFunc,activationFunc,lossWeight,hiddenLayers,df,C,dirPath,plotOn,channels=['94','131','171','193','211','304','335'],solarTypes=['AR','QS']):
    
    # Test Model
    removedNoise_image, addedNoise_image, denoiseTest_image = testModelArtificial(trainedModel,cleanTest_NN,cleanTest_image,noiseTest_NN,noiseTest_image)

    # Make new channels varible based on lossWeights
    newChannels = [x for (x,y) in zip(channels,lossWeight) if y == 1 in lossWeight]
        
    ###
    # Save generated images
    ###
    if plotOn == 1:
        
        cmaps=['sdoaia94','sdoaia131','sdoaia171','sdoaia193','sdoaia211','sdoaia304','sdoaia335']
        # Make new cmap varible based on lossWeights
        plotCmaps = [x for (x,y) in zip(cmaps,lossWeight) if y == 1 in lossWeight]
        
        # Make subdirectory
        subdirName = lossFunc+'_'+activationFunc+'_'+str(lossWeight)+'_'+str(hiddenLayers)+'_Artificial'
        subdirPath = dirPath+'/'+subdirName
        if os.path.exists(subdirPath) == False:
            os.mkdir(subdirPath)

        ### Plot all images (zoomed in)
        zoomStart_x_AR = 1100 
        zoomStart_y_AR = 500 
        zoomStart_x_QS = 200
        zoomStart_y_QS = 500 
        zoomSize = 150
        names = ['AR $I\'$','$A\'$','$\hat{A}\'$','QS $I$','$A$','$\hat{A}\'$']
        #stacks = [rescale(np.stack(cleanTest_image)[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(noiseTest_image)[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(cleanTest_image)[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(noiseTest_image)[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize]]
        stacks = [rescale(np.stack(cleanTest_image)[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(np.stack(noiseTest_image)[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(np.stack(cleanTest_image)[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(np.stack(noiseTest_image)[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize]]
        file_name = 'denoise_zoomed-'+subdirName
        histName = ['$I\'$','$A\'4','$\hat{A}\'$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath)

        ### Plot all images (very zoomed in)
        zoomSize = 50
        names = ['AR $I\'$','$A\'$','$\hat{A}\'$','QS $I$','$A$','$\hat{A}\'$']
        #stacks = [rescale(np.stack(cleanTest_image)[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(noiseTest_image)[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(cleanTest_image)[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(noiseTest_image)[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize]]
        stacks = [rescale(np.stack(cleanTest_image)[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(np.stack(noiseTest_image)[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(np.stack(cleanTest_image)[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(np.stack(noiseTest_image)[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize]]
        file_name = 'denoise_very_zoomed-'+subdirName
        histName = ['$I\'$','$A\'$','$\hat{A}\'$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath)
        
        ### Plot all images
        names = ['AR $I\'$','$A\'$','$\hat{A}\'$','QS $I$','$A$','$\hat{A}\'$']
        #stacks = [rescale(np.stack(cleanTest_image)[0],scaleStack),rescale(np.stack(noiseTest_image)[0],scaleStack),rescale(denoiseTest_image[0],scaleStack),
        #          rescale(np.stack(cleanTest_image)[1],scaleStack),rescale(np.stack(noiseTest_image)[1],scaleStack),rescale(denoiseTest_image[1],scaleStack)]
        stacks = [rescale(np.stack(cleanTest_image)[0],scaleStack),rescale(np.stack(noiseTest_image)[0],scaleStack),rescale(denoiseTest_image[0],scaleStack),
                  rescale(np.stack(cleanTest_image)[1],scaleStack),rescale(np.stack(noiseTest_image)[1],scaleStack),rescale(denoiseTest_image[1],scaleStack)]
        file_name = 'denoise-'+subdirName
        histName = ['$I\'$','$A\'$','$\hat{A}\'$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath)

        ### Plot all noise
        names = ['AR $P$','$(A\'-\hat{A}\')$','$(A\'-I\')$','QS $P$','$(A\'-\hat{A}\')$','$(A\'-I\')$']
        #stacks = [level_1_testNoise[0],rescale(removedNoise_image[0],scaleStack),rescale(addedNoise_image[0],scaleStack),
        #          level_1_testNoise[1],rescale(removedNoise_image[1],scaleStack),rescale(addedNoise_image[1],scaleStack)]
        stacks = [level_1_testNoise[0],rescale(removedNoise_image[0],scaleStack),rescale(addedNoise_image[0],scaleStack),
                  level_1_testNoise[0],rescale(removedNoise_image[0],scaleStack),rescale(addedNoise_image[0],scaleStack)]
        file_name = 'noise-'+subdirName
        histName = ['$P$','$(A\'-\hat{A}\')$','$(A\'-I\')$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath,cmaps=['seismic','seismic','seismic','seismic','seismic','seismic','seismic'])
    
    
        ### Plot zoomed in all noise
        names = ['AR $P$','$(A\'-\hat{A}\')$','$(A\'-I\')$','QS $P$','$(A\'-\hat{A}\')$','$(A\'-I\')$']
        #stacks = [level_1_testNoise[0][:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(addedNoise_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          level_1_testNoise[1][:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(removedNoise_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(addedNoise_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize]]
        stacks = [level_1_testNoise[0][:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(addedNoise_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  level_1_testNoise[0][:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(addedNoise_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize]]
        file_name = 'noise_zoomed-'+subdirName
        histName = ['$P$','$(A\'-\hat{A}\')$','$(A\'-I\')$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath,cmaps=['seismic','seismic','seismic','seismic','seismic','seismic','seismic'])
    ###
    # Update statisitcs dataframe
    ###
    for i in range(len(channels)):
        for j in range(len(solarTypes)):
            #if lossWeight[j] == 1:
            # Put everything in O,N,D notation and postprocess(rescale)
            aN = addedNoise_image[j][i].flatten()
            rN = removedNoise_image[j][i].flatten()
            D = denoiseTest_image[j][i].flatten()
            O = np.stack(cleanTest_image)[j][i].flatten()
            N = np.stack(noiseTest_image)[j][i].flatten()

            ### 
            # Statistics
            ###

            ### Added vs. Removed Noise 
            MSE_aNrN = np.mean(np.square(aN-rN)) # Mean Square Error
            L1_loss_aNrN = np.mean(np.abs(aN-rN)) 
            chi2_aNrN = getChi2(aN,rN)

            ### Original, Noisey, Denoised, Added, Removed 
            M1_O = np.mean(O) # First moment
            M2_O = np.std(O) # Second moment
            M1_N = np.mean(N) 
            M2_N = np.std(N) 
            M1_D = np.mean(D) 
            M2_D = np.std(D) 
            M1_aN = np.mean(aN) 
            M2_aN = np.std(aN) 
            M1_rN = np.mean(rN) 
            M2_rN = np.std(rN) 

            ### Original vs. Noisey & Denoised 
            #MSE_OD = np.mean(np.square(O-D))
            #L1_loss_OD = np.mean(np.abs(O-D)) 
            #chi2_OD = getChi2(O,N)
            #MSE_ON = np.mean(np.square(O-N))
            #L1_loss_ON = np.mean(np.abs(O-N)) 
            #chi2_ON = getChi2(O,N)

            configData = [lossFunc,activationFunc,str(lossWeight),str(hiddenLayers),channels[i],solarTypes[j]]
            data = [MSE_aNrN,L1_loss_aNrN,chi2_aNrN,
                    M1_O,M2_O,M1_N,M2_N,M1_D,M2_D,M1_aN,M2_aN,M1_rN,M2_rN]
            df_intermediate = pd.DataFrame([configData+data],columns=C)
            df = df.append(df_intermediate)       
            
    return df

<>:32: DeprecationWarning: invalid escape sequence \h
<>:32: DeprecationWarning: invalid escape sequence \h
<>:46: DeprecationWarning: invalid escape sequence \h
<>:53: DeprecationWarning: invalid escape sequence \h
<>:53: DeprecationWarning: invalid escape sequence \h
<>:67: DeprecationWarning: invalid escape sequence \h
<>:73: DeprecationWarning: invalid escape sequence \h
<>:73: DeprecationWarning: invalid escape sequence \h
<>:79: DeprecationWarning: invalid escape sequence \h
<>:85: DeprecationWarning: invalid escape sequence \h
<>:85: DeprecationWarning: invalid escape sequence \h
<>:91: DeprecationWarning: invalid escape sequence \h
<>:98: DeprecationWarning: invalid escape sequence \h
<>:98: DeprecationWarning: invalid escape sequence \h
<>:112: DeprecationWarning: invalid escape sequence \h
<>:32: DeprecationWarning: invalid escape sequence \h
<>:32: DeprecationWarning: invalid escape sequence \h
<>:46: DeprecationWarning: invalid escape sequence \h
<>:53: DeprecationWarning: 

In [175]:
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)
def runTestingUnmodified(trainedModel,lossFunc,activationFunc,lossWeight,hiddenLayers,df,C,dirPath,plotOn,channels=['94','131','171','193','211','304','335'],solarTypes=['AR','QS']):
    
    # Test Model with recent data
    removedNoise_image, denoiseTest_image = testModelUnmodified(trainedModel,cleanTestRecent_NN,cleanTestRecent_image)
    
    # Make new channels varible based on lossWeights
    newChannels = [x for (x,y) in zip(channels,lossWeight) if y == 1 in lossWeight]
        
    ###
    # Save generated images
    ###
    if plotOn == 1:
        
        cmaps=['sdoaia94','sdoaia131','sdoaia171','sdoaia193','sdoaia211','sdoaia304','sdoaia335']
        # Make new cmap varible based on lossWeights
        plotCmaps = [x for (x,y) in zip(cmaps,lossWeight) if y == 1 in lossWeight]
        
        # Make subdirectory
        subdirName = lossFunc+'_'+activationFunc+'_'+str(lossWeight)+'_'+str(hiddenLayers)+'_Unmodified'
        subdirPath = dirPath+'/'+subdirName
        if os.path.exists(subdirPath) == False:
            os.mkdir(subdirPath)

        ### Plot all images (zoomed in)
        zoomStart_x_AR = 900
        zoomStart_y_AR = 500 
        zoomStart_x_QS = 200
        zoomStart_y_QS = 1200
        zoomSize = 150
        names = ['AR $I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$','QS $I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$']
        #stacks = [rescale(np.stack(cleanTestRecent_image)[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(cleanTestRecent_image)[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(removedNoise_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize]]
        stacks = [rescale(np.stack(cleanTestRecent_image)[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(np.stack(cleanTestRecent_image)[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize]]
        file_name = 'denoise_zoomed-'+subdirName
        histName = ['$I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath,testType = 0)

        ### Plot all images (very zoomed in)
        zoomSize = 50
        names = ['AR I\'','$\hat{I}$\'','(I\'-$\hat{I}$\')','QS I\'','$\hat{I}$\'','(I\'-$\hat{I}$\')']
        #stacks = [rescale(np.stack(cleanTestRecent_image)[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(np.stack(cleanTestRecent_image)[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(denoiseTest_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize],
        #          rescale(removedNoise_image[1],scaleStack)[:,zoomStart_x:zoomStart_x+zoomSize,zoomStart_y:zoomStart_y+zoomSize]]
        stacks = [rescale(np.stack(cleanTestRecent_image)[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x_AR:zoomStart_x_AR+zoomSize,zoomStart_y_AR:zoomStart_y_AR+zoomSize],
                  rescale(np.stack(cleanTestRecent_image)[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(denoiseTest_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize],
                  rescale(removedNoise_image[0],scaleStack)[:,zoomStart_x_QS:zoomStart_x_QS+zoomSize,zoomStart_y_QS:zoomStart_y_QS+zoomSize]]
        file_name = 'denoise_very_zoomed-'+subdirName
        histName = ['$I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath,testType = 0)
        
        ### Plot all images
        names = ['AR $I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$','QS $I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$']
        #stacks = [rescale(np.stack(cleanTestRecent_image)[0],scaleStack),
        #          rescale(denoiseTest_image[0],scaleStack),
        #          rescale(removedNoise_image[0],scaleStack),
        #          rescale(np.stack(cleanTestRecent_image)[1],scaleStack),
        #          rescale(denoiseTest_image[1],scaleStack),
        #          rescale(removedNoise_image[1],scaleStack)]
        stacks = [rescale(np.stack(cleanTestRecent_image)[0],scaleStack),
                  rescale(denoiseTest_image[0],scaleStack),
                  rescale(removedNoise_image[0],scaleStack),
                  rescale(np.stack(cleanTestRecent_image)[1],scaleStack),
                  rescale(denoiseTest_image[1],scaleStack),
                  rescale(removedNoise_image[1],scaleStack)]
        file_name = 'denoise-'+subdirName
        histName = ['$I\'$','$\hat{I}\'$','$(I\'-\hat{I}\')$']
        saveFigure = 1
        bins = 50
        stackPlotter(names,stacks,file_name,histName,saveFigure,bins,subdirPath,testType = 0)

    ###
    # Update statisitcs dataframe
    ###
    for i in range(len(channels)):
        for j in range(len(solarTypes)):
            #if lossWeight[j] == 1:
            # Put everything in O,N,D notation and postprocess(rescale)
            rN = removedNoise_image[j][i].flatten()
            D = denoiseTest_image[j][i].flatten()
            O = np.stack(cleanTestRecent_image)[j][i].flatten()

            ### 
            # Statistics
            ###

            ### Original vs. Denoise
            MSE_OD = np.mean(np.square(O-D)) # Mean Square Error
            L1_loss_OD = np.mean(np.abs(O-D)) 
            chi2_OD = getChi2(O,D)

            ### Original, Noisey, Denoised, Added, Removed 
            M1_O = np.mean(O) # First moment
            M2_O = np.std(O) # Second moment
            M1_D = np.mean(D) 
            M2_D = np.std(D) 
            M1_rN = np.mean(rN) 
            M2_rN = np.std(rN) 

            configData = [lossFunc,activationFunc,str(lossWeight),str(hiddenLayers),channels[i],solarTypes[j]]
            data = [MSE_OD,L1_loss_OD,chi2_OD,
                    M1_O,M2_O,M1_D,M2_D,M1_rN,M2_rN]
            df_intermediate = pd.DataFrame([configData+data],columns=C)
            df = df.append(df_intermediate)       
    
    return df

In [179]:
# Main()
import os
import pandas as pd

### Configurations
#batch_size = 5
batch_size = 25
epochs = 25
#epochs = 15
learningRate = 0.0001
plotOn = 1
importDF = 0
regCoeff = 0 #Regularization coefficent for activations on hidden layers (used for sparse autoencoder)

activationFunctions = ['LeakyReLU'] # Options are ReLU, LeakyReLU, Linear

lossWeights =[[1,1,1,1,1,1,1]]

hiddenLayers = [[6,4,6]]

lossFunctions = ['L1'] # Options are MSE & L1

path = '/sanhome/penwarden/public_html/'
dirName = '13th_Testing_overnight_B25_E25'
dirPath = path+dirName
testType = 2 # 2 for both, 1 is for the artificially noisy images, 0 is for the unmodified recent images
###

# Create Test directory 
if os.path.exists(dirPath) == False:
    os.mkdir(dirPath)

layersString = []
for i in range(len(hiddenLayers)):
    layersString.append(str(hiddenLayers[i]))


    # Initialize dataframe
    configList = ['Loss Function']+['Activation Function']+['Loss Weights']+['Hidden Layers Config']+['Channel']+['Solar Type']

    if testType == 1 or testType == 2:
        statisticsList_artificial = ['MSE_aNrN','L1_loss_aNrN','Chi2_aNrN',
                        'M1_O','M2_O','M1_N','M2_N','M1_D','M2_D','M1_aN','M2_aN','M1_rN','M2_rN']
        C_artificial = pd.Index(configList+statisticsList_artificial)
    if testType == 0 or testType == 2:
        statisticsList_unmodified = ['MSE_OD','L1_loss_OD','Chi2_OD',
                        'M1_O','M2_O','M1_D','M2_D','M1_rN','M2_rN']
        C_unmodified = pd.Index(configList+statisticsList_unmodified)
    
if importDF == 0:
    if testType == 1 or testType == 2:
        df_artificial = pd.DataFrame(columns=C_artificial)
    if testType == 0 or testType == 2:
        df_unmodified = pd.DataFrame(columns=C_unmodified)
else: 
    if testType == 1 or testType == 2:
        df_artificial = pd.read_pickle(dirPath+'/'+dirName+'_artificial.pkl')
    if testType == 0 or testType == 2:
        df_unmodified = pd.read_pickle(dirPath+'/'+dirName+'_unmodified.pkl')

for i in range(len(lossFunctions)):
    for j in range(len(activationFunctions)):
        for k in range(len(lossWeights)):
            for l in range(len(hiddenLayers)):
                trainedModel = runTraining(lossFunctions[i],activationFunctions[j],lossWeights[k],hiddenLayers[l],learningRate,batch_size,epochs,regCoeff,dirPath)
                if testType == 1 or testType == 2:
                    df_artificial = runTestingArtificial(trainedModel,lossFunctions[i],activationFunctions[j],lossWeights[k],hiddenLayers[l],df_artificial,C_artificial,dirPath,plotOn)
                if testType == 0 or testType == 2:
                    df_unmodified = runTestingUnmodified(trainedModel,lossFunctions[i],activationFunctions[j],lossWeights[k],hiddenLayers[l],df_unmodified,C_unmodified,dirPath,plotOn)
             
if testType == 1 or testType == 2:
    # Make saves
    dfUnstyled_artificial = df_artificial.copy()
    dfUnstyled_artificial.to_pickle(dirPath+'/'+dirName+'_artificial.pkl') # Save the dataframe without styler for future use
    dfUnstyled_artificial.to_csv(dirPath+'/'+dirName+'_artificial.csv') # Save csv just in case

    # Make Hierarchical Indices
    df_artificial = df_artificial.set_index(['Loss Function','Activation Function','Loss Weights','Hidden Layers Config','Channel','Solar Type'])
    df_artificial = df_artificial.replace([np.inf, -np.inf], np.nan)

    # Add color
    df_artificial = df_artificial.style.apply(colorDataFrameArtificial,axis=None)\
    .background_gradient('RdYlGn_r',axis=0,subset=['MSE_aNrN'], vmin = 0, vmax = 2, text_color_threshold = 0)\
    .background_gradient('RdYlGn_r',axis=0,subset=['L1_loss_aNrN'], vmin = 0, vmax = 2,text_color_threshold = 0)\
    .background_gradient('RdYlGn_r',axis=0,subset=['Chi2_aNrN'], vmin = 0, vmax = 100, text_color_threshold = 0)
        
    # Write to file
    df_artificial.to_excel(dirPath+'/'+dirName+'_artificial.xlsx',float_format='%.3f') # Write to excel file to retain dataframe hierarchical indexing and coloring
    with open(dirPath+'/'+dirName+'_artificial.html', 'w') as fo:
        fo.write(df_artificial.render())
        
if testType == 0 or testType == 2:  
    # Make saves
    dfUnstyled_unmodified = df_unmodified.copy()
    dfUnstyled_unmodified.to_pickle(dirPath+'/'+dirName+'_unmodified.pkl') # Save the dataframe without styler for future use
    dfUnstyled_unmodified.to_csv(dirPath+'/'+dirName+'_unmodified.csv') # Save csv just in case

    # Make Hierarchical Indices
    df_unmodified = df_unmodified.set_index(['Loss Function','Activation Function','Loss Weights','Hidden Layers Config','Channel','Solar Type'])
    df_unmodified = df_unmodified.replace([np.inf, -np.inf], np.nan)

    # Add color
    df_unmodified = df_unmodified.style.apply(colorDataFrameUnmodified,axis=None)
    
    # Write to file
    df_unmodified.to_excel(dirPath+'/'+dirName+'_unmodified.xlsx',float_format='%.3f') # Write to excel file to retain dataframe hierarchical indexing and coloring
    with open(dirPath+'/'+dirName+'_unmodified.html', 'w') as fo:
        fo.write(df_unmodified.render())
        
print('DONE')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 7)            56          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 6)            48          dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 4)            28          dense_1[0][0]                    
______________________________________________________________________________________________

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


DONE
